In [ ]:
# data_ingest.py
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# ==================== CONFIGURATION ====================
DELTA_TABLE_NAME = "house_price_delta"

# Global variables to hold data/features for the next script (in a Databricks Notebook environment)
# In a true MLOps pipeline, you would save this to another Delta table or a file (e.g., /tmp/data.parquet)
global TRAINING_DATA_DF
global FEATURE_COLUMNS
TRAINING_DATA_DF = None
FEATURE_COLUMNS = None 

# ==================== FUNCTIONS ====================

def initialize_spark():
    """SparkSession को इनिशियलाइज़ करता है (यदि पहले से नहीं है)"""
    # Databricks में, SparkSession पहले से ही 'spark' variable में उपलब्ध होती है।
    if 'spark' in globals() and isinstance(globals()['spark'], SparkSession):
        return globals()['spark']
    try:
        spark = SparkSession.builder.appName("DataIngest").getOrCreate()
        return spark
    except Exception as e:
        print(f"SparkSession initialization failed: {e}")
        return None

def ingest_data(spark: SparkSession, table_name: str):
    """
    डेल्टा टेबल से हाउस प्राइस डेटा को इंगेस्ट करता है और फीचर्स/लेबल को अलग करता है।
    """
    if spark is None:
        print("❌ SparkSession उपलब्ध नहीं है। डेटा इंगेस्ट नहीं किया जा सकता।")
        return None, None
        
    print(f"डेल्टा टेबल '{table_name}' से डेटा लोड हो रहा है...")

    try:
        # डेल्टा टेबल से डेटा पढ़ें
        house_df = spark.read.format("delta").table(table_name)
        
        feature_cols = ['sq_feet', 'num_bedrooms', 'num_bathrooms', 'year_built', 'location_score']
        label_col = 'price'
        
        # फ़ीचर्स (Features) और लेबल (Label) को अलग करें (price को label के रूप में)
        ingested_df = house_df.select(*feature_cols, col(label_col).alias("label"))
        
        print(f"डेटा सफलतापूर्वक इंगेस्ट हुआ। कुल पंक्तियाँ: {ingested_df.count()}")
        ingested_df.printSchema()
        
        return ingested_df, feature_cols
        
    except Exception as e:
        print(f"डेटा इंगेस्ट में त्रुटि: {e}")
        return None, None

# ==================== EXECUTION ====================

if __name__ == "__main__":
    spark_session = initialize_spark()
    ingested_data_df, features = ingest_data(spark_session, DELTA_TABLE_NAME)

    if ingested_data_df is not None:
        # ग्लोबल वेरिएबल में सेव करें ताकि ट्रेनिंग स्क्रिप्ट इसका उपयोग कर सके
        # (ध्यान दें: Databricks Notebooks में, आप MAGIC COMMAND्स जैसे %run या job parameters का उपयोग करेंगे)
        globals()['TRAINING_DATA_DF'] = ingested_data_df
        globals()['FEATURE_COLUMNS'] = features
        print("✅ डेटा को ग्लोबल वेरिएबल्स में सेव किया गया।")
    else:
        print("❌ डेटा इंगेस्ट विफल रहा।")